In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

In [ ]:
results = pd.read_csv("../results.csv")

In [ ]:
results = results.sort_values(by=["concurrency", "framework"])

In [ ]:
results = results[results["framework"] != "express_without_cluster"]
results = results[results["framework"] != "express_big_instance_without_cluster"]
results = results[results["framework"] != "express_big_instance_with_cluster"]
results = results[results["framework"] != "spring_large_instance"]

In [ ]:
max_requests_per_second_spring = results[results["framework"] == "spring"].groupby("endpoint")["requests_per_second"].max()
max_requests_per_second_express = results[results["framework"] == "express_with_cluster"].groupby("endpoint")["requests_per_second"].max()
max_requests_per_second_django = results[results["framework"] == "django"].groupby("endpoint")["requests_per_second"].max()

max_requests_per_second_spring_get_price_endpoint = max_requests_per_second_spring["get_price"]
max_requests_per_second_express_get_price_endpoint = max_requests_per_second_express["get_price"]
max_requests_per_second_django_get_price_endpoint = max_requests_per_second_django["get_price"]

In [ ]:
ec2_instance_price_per_second = 0.12075 / 3600

In [ ]:
fig = plt.figure(figsize=(6, 3), dpi=200)
plt.plot(range(1, 10001), [(ec2_instance_price_per_second * (math.ceil(i / max_requests_per_second_django_get_price_endpoint))) / i for i in range(1, 10001)], label="Django")
plt.plot(range(1, 10001), [(ec2_instance_price_per_second * (math.ceil(i / max_requests_per_second_express_get_price_endpoint))) / i for i in range(1, 10001)], label="Express")
plt.plot(range(1, 10001), [(ec2_instance_price_per_second * (math.ceil(i / max_requests_per_second_spring_get_price_endpoint))) / i for i in range(1, 10001)], label="Spring")

handles, labels = plt.gca().get_legend_handles_labels()
order = [2, 1, 0]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])

plt.ylim(top=0.0004 / 3600, bottom=0)
plt.xlabel("Requests per second")
plt.ylabel("Cost per request (USD)")

fig_dir = "../graphs/framework_price_comparison/"
os.makedirs(fig_dir, exist_ok=True)
plt.savefig(fig_dir + "/get_price_price_per_request.png",dpi=fig.dpi, bbox_inches = 'tight')
plt.show()
plt.close()